SIMPLE GANS

In [ ]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from  torch.distributions.multivariate_normal import MultivariateNormal
from torch.optim import lr_scheduler
from tqdm import tqdm
from IPython import display

import torchvision.utils as vutils
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
from torchvision.transforms import ToPILImage


import os
from os import listdir
from pathlib import Path
import imghdr
import skillsnetwork

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-GPXX0A52EN/G_trained.pth
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-GPXX0A52EN/D_trained.pth

In [ ]:
# This function will allow us to easily plot data taking in x values, y values, and a title
def plot_distribution(real_data,generated_data,discriminator=None,density=True):
    
    plt.hist(real_data.numpy(), 100, density=density, facecolor='g', alpha=0.75, label='real data')
    plt.hist(generated_data.numpy(), 100, density=density, facecolor='r', alpha=0.75,label='generated data q(z) ')
    
    if discriminator:
        max_=torch.max(real_data.max(),generated_data.max().detach())
        min_=torch.min(real_data.min(),generated_data.min().detach())
        x=torch.linspace(start=min_, end=max_, steps=100)
        plt.plot(x.numpy(),discriminator(x.view(-1,1)).detach().view(-1).numpy(),label='discriminator',color='k')
        plt.plot(x.numpy(),0.5*np.ones(x.shape),label='0.5',color='b')
        plt.legend()
        plt.show()

In [ ]:
# plot batch of images
def plot_image_batch(my_batch):

  fig, axes = plt.subplots(nrows=8, ncols=8, figsize=(10, 10))
  img_num=0
  for i in range(8):
      for j in range(8):
          ax = axes[i][j]
          img_num+=1
  
          ax.imshow(np.transpose(vutils.make_grid(my_batch[img_num].to(device), padding=2, normalize=True).cpu(),(1,2,0)))
  plt.show()

In [ ]:
def get_accuracy(X,Xhat):
    total=0
    py_x=D(X)
    total=py_x.mean()
    py_x=D(Xhat)
    total+=py_x.mean()
    return total/2

In [ ]:
class Dataset(Dataset):
    def __init__(self, covariance_matrix, mean,n_samples=1000):
        self.obj =  MultivariateNormal(loc=mean, covariance_matrix=covariance_matrix)
        self.mean=mean
        self.covariance_matrix=covariance_matrix
        self.X=torch.tensor([[self.obj.sample()] for n in range(n_samples)])
        
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        
        return self.X[idx,:]

In [ ]:
n_samples=10000
mean=10*torch.ones(1)
covariance_matrix=0.1*torch.eye(1)

dataset=Dataset(covariance_matrix=covariance_matrix, mean=mean,n_samples=n_samples)

In [ ]:
Z=torch.randn(n_samples,1)

In [ ]:
print("mean:",Z.mean())
print("standard deviation:",Z.std())

In [ ]:
Xhat=Z+10

In [ ]:
print("mean:",Xhat.mean())
print("standard deviation:",Xhat.std())

In [ ]:
plot_distribution(real_data=dataset.X,generated_data=Xhat)

In [ ]:
class Generator(nn.Module):
    def __init__(self,input_dim=1):
        super(Generator,self).__init__()
        self.l1=nn.Linear(1,input_dim)
    
    def forward(self, x):
        return self.l1(x)
G=Generator()

In [ ]:
class Discriminator(nn.Module):
    def __init__(self,input_dim=1):
        super(Discriminator,self).__init__()
        self.l1=nn.Linear(1,input_dim)
    
    def forward(self, x):
        return torch.sigmoid(self.l1(x))

D=Discriminator() 

In [ ]:
D

In [ ]:
n_samples=1000
Xhat=G(torch.randn(n_samples,1))
plot_distribution(real_data=dataset.X,generated_data=Xhat.detach(),discriminator=D)

In [ ]:
py_x=D(dataset.X)
torch.sum(py_x>0.5)/len(py_x)

In [ ]:
py_x=D(Z)
torch.sum(py_x>0.5)/len(py_x)

In [ ]:
Z

In [ ]:
get_accuracy(dataset.X,Xhat)

In [ ]:
criterion=nn.BCELoss()

In [ ]:
criterion(D(G(Z)),torch.zeros(Z.shape[0],1))

In [ ]:
criterion(D(dataset.X),torch.ones(len(dataset),1))+criterion(D(G(Z)),torch.zeros(Z.shape[0],1))/2

In [ ]:
# Learning rate for optimizers
lr = 0.1

# Beta1 hyperparam for Adam optimizers
beta1 = 0.1

G=Generator()
D=Discriminator()

optimizerG=optim.Adam(G.parameters(),lr=lr,betas=(beta1, 0.999))
optimizerD=optim.Adam(D.parameters(),lr=lr,betas=(beta1, 0.999))

In [ ]:
batch_size=100

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True)

In [ ]:
LOSS_G=[]
LOSS_D=[]

epochs=20

noise_dim=1
epsilon=100 

# Training loop
for epoch in tqdm(range(epochs)):
    # Plot the data distribution and generated samples
    Xhat=G(torch.randn(n_samples,1))
    plot_distribution(real_data=dataset.X, generated_data=Xhat.detach(), discriminator=D)
    
    # Training the discriminator
    for real_data in dataloader:
        noise = torch.randn(batch_size, 1)
        fake_data = G(noise)
        
        # Discriminator predictions for real and fake data
        real_predictions = D(real_data)
        fake_predictions = D(fake_data)
 
        # Discriminator loss for real and fake data
        loss_D_real = criterion(real_predictions, torch.ones(batch_size, 1))
        loss_D_fake = criterion(fake_predictions, torch.zeros(batch_size, 1))
        
        # Overall discriminator loss
        loss_D = (loss_D_fake + loss_D_real) / 2
        LOSS_D.append(loss_D.detach().item())
        
        # Backpropagation and optimizer update for discriminator
        D.zero_grad()
        loss_D.backward(retain_graph=True)
        optimizerD.step()
        
        # Training the generator
        output = D(fake_data)
        loss_G = criterion(output, torch.ones(batch_size, 1))
        LOSS_G.append(loss_G.detach().item())
    
        # Backpropagation and optimizer update for generator
        G.zero_grad()
        loss_G.backward()
        optimizerG.step()
    
    # Save and display the generator and discriminator if the performance increases 
    Xhat = G(torch.randn(len(dataset), 1))
    accuracy = abs(0.5 - get_accuracy(dataset.X, Xhat))
    if accuracy < epsilon:
        epsilon = accuracy
        torch.save(D.state_dict(), 'D.pth')
        torch.save(G.state_dict(), 'G.pth')
        print("Epoch:", epoch)

In [ ]:
D=Discriminator()
D.load_state_dict(torch.load("D.pth"))
G=Generator()
G.load_state_dict(torch.load("G.pth"))

Xhat=G(torch.randn(len(dataset), 1))
plot_distribution(real_data=dataset.X, generated_data=Xhat.detach(), discriminator=D)

DEEP CONV GANS

In [ ]:
img_height, img_width, batch_size=64,64,128

In [ ]:
current_directory = os.getcwd()
directory=os.path.join(current_directory ,'cartoon_20000')
[filename for filename in os.listdir(directory) if filename.endswith('.jpg') ]

class Dataset(Dataset):
    def __init__(self, transform=None):
      current_directory = os.getcwd()
      directory=os.path.join(current_directory ,'cartoon_20000')

      self.file_paths = [os.path.join(directory,filename ) for filename in os.listdir(directory) if filename.endswith('.jpg') ]
      self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        image_path = self.file_paths[index]
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image

In [ ]:
dataset=Dataset()

In [ ]:
for i in range(1,5):
  plt.imshow(dataset[i])
  plt.show()

In [ ]:
image_size = 64
transform=transforms.Compose([
                               transforms.Resize((64, 64)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])

In [ ]:
dataset=Dataset(transform=transform)

In [ ]:
# Create the dataloader
# batch_size Batch size during training
batch_size = 128
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=True)

In [ ]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
device

In [ ]:
real_batch = next(iter(dataloader))
real_batch.shape

In [ ]:
plot_image_batch(real_batch)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Block 1:input is Z, going into a convolution
            nn.ConvTranspose2d(latent_vector_size, 64 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.ReLU(True),
            # Block 2: input is (64 * 8) x 4 x 4
            nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.ReLU(True),
            # Block 3: input is (64 * 4) x 8 x 8
            nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(True),
            # Block 4: input is (64 * 2) x 16 x 16
            nn.ConvTranspose2d(64 * 2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # Block 5: input is (64) x 32 x 32
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output: output is (3) x 64 x 64
        )
 
    def forward(self, input):
        output = self.main(input)
        return output    

In [ ]:
latent_vector_size=128

G = Generator().to(device)
print(G)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Block 1: input is (3) x 64 x 64
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 2: input is (64) x 32 x 32
            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 3: input is (64*2) x 16 x 16
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 4: input is (64*4) x 8 x 8
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 5: input is (64*8) x 4 x 4
            nn.Conv2d(64 * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),
            nn.Flatten()
            # Output: 1
        )
 
    def forward(self, input):
        output = self.main(input)
        return output

In [ ]:
D =Discriminator().to(device)
print(D)

In [ ]:
learning_rate = 0.0002
G_optimizer = optim.Adam(G.parameters(), lr = learning_rate, betas=(0.5, 0.999))
D_optimizer = optim.Adam(D.parameters(), lr = learning_rate, betas=(0.5, 0.999))
scheduler_G = lr_scheduler.StepLR(G_optimizer, step_size=10, gamma=0.1)
scheduler_D = lr_scheduler.StepLR(D_optimizer, step_size=10, gamma=0.1)
device

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias) 

D.apply(weights_init)
G.apply(weights_init)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
LOSS_G = []
LOSS_D = []
epochs = 1
epsilon = 100

# Training loop
for epoch in tqdm(range(epochs)):
    print(epoch)
    for real_data in dataloader:
        real_data = real_data.to(device)
        noise =torch.randn(batch_size  , latent_vector_size , 1, 1, device=device)
        fake_data = G(noise)
        
        # Discriminator predictions for real and fake data
        real_predictions = D(real_data)
        fake_predictions = D(fake_data)
 
        # Discriminator loss for real and fake data
        loss_D_real = criterion(real_predictions, torch.ones(len(real_predictions), 1).to(device))
        loss_D_fake = criterion(fake_predictions, torch.zeros(len(fake_predictions), 1).to(device))
        
        # Overall discriminator loss
        loss_D = (loss_D_fake + loss_D_real) / 2
        LOSS_D.append(loss_D.detach().item())
        
        # Backpropagation and optimizer update for discriminator
        D.zero_grad()
        loss_D.backward(retain_graph=True)
        D_optimizer.step()
        
        # Training the generator
        output = D(fake_data)
        loss_G = criterion(output, torch.ones(len(output), 1).to(device))
        LOSS_G.append(loss_G.detach().item())
    
        # Backpropagation and optimizer update for generator
        G.zero_grad()
        loss_G.backward()
        G_optimizer.step()
    
    # Using LR Scheduler
    scheduler_G.step()
    scheduler_D.step()
    
    # Displaying Images
    Xhat = G(noise).to(device).detach()
    plot_image_batch(Xhat)
    print("Epoch:", epoch)
    
    # Saving the model
    torch.save(D.state_dict(), 'D.pth')
    torch.save(G.state_dict(), 'G.pth')